In [1]:
import ray.tune
from pathlib import Path
from dataclasses import dataclass
from functools import lru_cache
import pandas as pd
from typing import List

/home/ledoyen202/Documents/labo/etr-peft-composition/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-01 22:56:55,847	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-01 22:56:56,341	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [99]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
from etr_fr_expes import metric

2025-04-01 22:57:00,903	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ledoyen202/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
EXPE_DIR = Path("../../experimentations/").resolve()

In [111]:
@dataclass
class Expe:
    expe_name: str
    metric: str
    mode: str
    adapter_name: str
    expe_dir: str = EXPE_DIR
    _expe_analysis = None

    @property
    def expe_analysis(self):
        hp_search_dir = self.expe_dir / self.expe_name / "results" / f"{self.expe_name}_hp_search"
        if self._expe_analysis is None:
            self._expe_analysis = ray.tune.ExperimentAnalysis(hp_search_dir)
        return self._expe_analysis

    @property
    def dataframe(self):
        dfs = self.expe_analysis.trial_dataframes
        res = (pd.concat(dfs, ignore_index=True)
            .reset_index(level=0)
            .assign(expe=self.expe_name)
            .sort_values(by=self.metric, ascending=self.mode == "min")
            .rename(columns=lambda x: x.replace(self.adapter_name, 'adapter'))
        )
        return res
    
    def get_texts_df(self, row_idx, _type="test"):
        row_df = self.dataframe.iloc[row_idx].filter(regex=f"{_type}.*texts").to_frame().T
        row_df = row_df.explode(list(row_df.columns)).reset_index(drop=True).stack().to_frame()
        return row_df
    
    @property
    def best_trial(self):
        return self.expe_analysis.get_best_trial(metric=self.metric, mode=self.mode, scope="all")

In [112]:
@dataclass
class Analysis:
    expes: List[Expe]

    @property
    def dataframe(self):
        dfs = [expe.dataframe for expe in self.expes]
        return pd.concat(dfs, ignore_index=True)

In [113]:
analysis = Analysis(
    expes=[
        Expe(expe_name="mbarthez.lora.etrfr", metric="eval_etr_fr_srb", mode="max", adapter_name="lora_etr_fr"),
        Expe(expe_name="mbarthez.lora.orangesum", metric="eval_orangesum_rougeL", mode="max", adapter_name="lora_orangesum"),
        Expe(expe_name="mbarthez.lora.wikilarge-fr", metric="eval_wikilarge_fr_sari", mode="max", adapter_name="lora_wikilarge_fr"),
        Expe(expe_name="mistral.lora.etrfr", metric="eval_etr_fr_srb", mode="max", adapter_name="lora_etr_fr"),
        Expe(expe_name="mistral.lora.orangesum", metric="eval_orangesum_rougeL", mode="max", adapter_name="lora_orangesum"),
        Expe(expe_name="mistral.lora.wikilarge", metric="eval_wikilarge_fr_sari", mode="max", adapter_name="lora_wikilarge"),
    ]
)

In [115]:
analysis.expes[-3].get_texts_df(2)

0
0  test_etr_fr_texts/inputs                                                                                                                                                                                                                                                                                                                                                                              La cloche sonne. C'est la fin de l'année, tant attendue par tous les élèves. Une ruée de jeunes franchit la grille du collège avec des hurlements de joie. Lucas traverse la cour d'un pas léger. « Salut, bonnes vacances! », lui crient ses amis en lui faisant des signes de la main. Bonnes vacances, pense Lucas, facile à dire, mais qu'est-ce que je vais bien pouvoir faire de mes journées? Comme le magasin de ses parents reste ouvert tout l'été, la famille ne part pas. Ses copains, eux, seront tous par monts et par vaux. Et, comble de malchance, Manon, la petite-fille des voisins, ne vient pas au village cet été. Elle a appelé Lucas de l'aéroport pour lui annoncer qu'elle ratait deux jours de classe pour partir pour l'Afrique avec ses parents. Elle avait l'air si contente de faire un safari-photo ! Lucas se réjouit pour elle, mais au fond, il est déçu de ne pas la revoir. Il doit bien l'admettre : Manon lui manque terriblement.
   test_etr_fr_texts/labels                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  La cloche sonne. L'école est finie. Super ! Ce sont les vacances. Mais Lucas se demande : « Que faire de mes vacances ? » Manon ne vient pas en vacances chez son papi et sa mamie. Manon fait un safari en Afrique avec son papa et sa maman. Lucas est triste de ne pas voir Manon.
   test_etr_fr_texts/predictions                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  La cloche sonne. C'est la fin de l'année. Les élèves sortent du collège. « Bonnes vacances ! » Les amis de Lucas lui font des signes de la main. « Bonnes vacances ! » Lucas pense : « Je ne sais pas ce que je vais faire de mes vacances. » La famille de Lucas ne part pas en vacances car le magasin demeure ouvert toute l'
1  te